In [ ]:
import pandas as pd
!pip install -U jax[cuda112] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -U jaxlib[cuda112] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install optax
!pip install dm-haiku
!pip install tensorflow-probability
!pip install tensorflow-datasets
!pip install imbalanced-learn
!pip install git+https://github.com/blackjax-devs/blackjax.git

In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "False"
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"] = "platform"

In [2]:
%cd /home/xabush/code/snet/moses-incons-pen-xp/notebooks/variable_selection/cancer/nn
import matplotlib.pyplot as plt
import tensorflow_probability.substrates.jax as tfp
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
tfd = tfp.distributions
from gibbs_sampler import *

plt.style.use('ggplot')
%load_ext autoreload

/home/xabush/code/snet/moses-incons-pen-xp/notebooks/variable_selection/cancer/nn


/home/xabush/miniconda3/lib/python3.9/site-packages/chex/_src/pytypes.py:37: FutureWarning: jax.tree_structure is deprecated, and will be removed in a future release. Use jax.tree_util.tree_structure instead.
  PyTreeDef = type(jax.tree_structure(None))


In [3]:
data_dir = "/home/xabush/code/snet/moses-incons-pen-xp/data"

In [5]:
tcga_gene_expr_df = pd.read_csv(f"{data_dir}/tcgaRNAseqEntrez.csv", index_col="sample")
tcga_gene_expr_df

,63916,90993,6205,9240,4313,118611,23051,2073,84059,51289,...,55127,51504,408263,5720,10734,201799,114792,254187,84787,7536
sample,,,,,,,,,,,,,,,,,,,,,
TCGA.A1.A0SD,0.5071,1.4345,0.7650,0.5260,1.5060,-2.0183,0.0433,-0.1072,0.1031,-0.0620,...,-1.1963,-0.5669,0.4770,0.0337,-2.3310,2.7098,-0.2495,-0.7820,-1.0585,-0.7914
TCGA.A1.A0SE,0.1814,0.8908,0.7160,0.1318,0.9578,-0.5903,-0.0437,-0.2385,-0.1522,0.4425,...,-0.2114,-0.6249,0.3810,0.3335,-2.2385,0.6798,-0.4920,-1.7955,-0.7757,-1.1238
TCGA.A1.A0SH,0.4615,2.2593,0.4171,0.3250,2.2855,-1.7268,0.4205,0.0072,-0.6960,0.2960,...,-0.9239,-0.0040,0.3005,0.1680,-1.7900,1.3068,-0.3722,-0.7135,-0.8375,-1.2906
TCGA.A1.A0SJ,0.8770,0.4378,0.1150,0.7578,-0.0893,-1.7837,0.2510,0.0352,-0.6406,0.6885,...,-1.0014,-0.4110,0.1905,0.7040,-2.2770,1.2083,1.2638,-1.3870,-1.2565,-0.8082
TCGA.A1.A0SK,1.4123,-0.6372,0.4929,0.9432,-1.2332,-2.7550,-1.3315,0.7910,0.0145,-0.0600,...,-2.2761,-0.1885,-0.4842,-1.3768,-0.4602,-0.4422,1.3678,-1.0530,0.2732,-0.1535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA.E2.A1B4,-0.3512,1.7183,0.3261,-0.3635,0.4007,-1.7263,0.6043,0.7085,1.2884,-0.2805,...,-1.4000,0.5351,0.8638,0.1140,-2.0697,2.8032,0.6227,0.1075,-0.6942,-0.5601
TCGA.E2.A1B5,-0.2104,-0.0718,0.6036,-1.2445,-1.0060,0.9515,-0.8528,1.2777,-1.0344,0.7400,...,-0.6232,0.5499,-0.0472,0.0658,0.9070,1.3512,-0.0490,-0.3145,-0.7007,0.2822
TCGA.E2.A1B6,-0.0371,1.4002,1.4066,-0.4212,-0.2172,-1.4045,-0.5692,0.9170,1.0480,-0.0150,...,-1.2822,0.4176,0.8300,0.5608,-0.4182,1.2095,0.0413,-0.7695,-0.8847,-0.7779


In [7]:
tcga_pheno_df = pd.read_csv(f"{data_dir}/tcgaPheno.csv", index_col="sample")
tcga_pheno_df

,years_to_birth,Tumor_purity,pathologic_stage,pathology_T_stage,pathology_N_stage,pathology_M_stage,histological_type,number_of_lymph_nodes,PAM50,ER.Status,PR.Status,HER2.Status,gender,radiation_therapy,race,ethnicity,Median_overall_survival,overall_survival,status,overallsurvival
sample,,,,,,,,,,,,,,,,,,,,
TCGA.A1.A0SD,59.0,0.6906,stageii,t2,n0,m0,infiltratingductalcarcinoma,0.0,LumA,NaN,NaN,NaN,female,NaN,white,nothispanicorlatino,0.0,437.0,0.0,"437,0"
TCGA.A1.A0SE,56.0,0.7979,stagei,t1,n0,m0,mixedhistology(pleasespecify),0.0,LumA,NaN,NaN,NaN,female,NaN,white,nothispanicorlatino,0.0,1321.0,0.0,"1321,0"
TCGA.A1.A0SH,39.0,0.6574,stageii,t2,n0,m0,infiltratingductalcarcinoma,0.0,LumA,NaN,NaN,NaN,female,NaN,white,nothispanicorlatino,0.0,1437.0,0.0,"1437,0"
TCGA.A1.A0SJ,39.0,0.7984,stageiii,t3,n1,m0,infiltratingductalcarcinoma,3.0,LumA,NaN,NaN,NaN,female,NaN,blackorafricanamerican,nothispanicorlatino,0.0,416.0,0.0,"416,0"
TCGA.A1.A0SK,54.0,0.9288,stageii,t2,n0,m0,"other,specify",0.0,Basal,NaN,NaN,NaN,female,NaN,asian,nothispanicorlatino,0.0,967.0,1.0,"967,1"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA.E2.A1B4,74.0,0.8854,stageiii,t1,n2,m0,infiltratingductalcarcinoma,3.0,LumA,NaN,NaN,NaN,female,yes,white,nothispanicorlatino,0.0,1004.0,1.0,"1004,1"
TCGA.E2.A1B5,46.0,0.3604,stageii,t2,n0,m0,infiltratinglobularcarcinoma,0.0,LumA,NaN,NaN,NaN,female,yes,white,nothispanicorlatino,0.0,984.0,0.0,"984,0"
TCGA.E2.A1B6,44.0,0.4396,stageii,t2,n0,m0,infiltratingductalcarcinoma,0.0,Basal,NaN,NaN,NaN,female,no,white,nothispanicorlatino,0.0,867.0,0.0,"867,0"


In [18]:
tcga_brca_df = pd.merge(tcga_gene_expr_df, tcga_pheno_df["status"], left_index=True, right_index=True)
tcga_brca_df =  tcga_brca_df[tcga_brca_df["status"].notna()]
tcga_brca_df.to_csv(f"{data_dir}/tcga_brca_entrez.csv")

In [19]:
na_columns = tcga_brca_df.columns[tcga_brca_df.isna().any()].to_list()
len(na_columns)

139

In [20]:
X, y  = tcga_brca_df[tcga_brca_df.columns.difference([na_columns] + ["status"])], tcga_brca_df["status"]

TypeError: unhashable type: 'list'

In [16]:
seed = 1234
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X, y, test_size=0.2, random_state=seed, shuffle=True, stratify=y)

ValueError: Input y contains NaN.

In [75]:
train_mean = X_train_df.mean()
train_std = X_train_df.std()

X_train_df = (X_train_df - train_mean) / train_std
X_test_df = (X_test_df - train_mean) / train_std

In [65]:
X_train_df

,7A5,A1BG,A2BP1,A2M,A4GALT,AAAS,AAK1,AANAT,AARS2,AASDH,...,hCG_18290,hCG_1990170,hCG_2001000,hCG_31916,psiTPTE22,tAKR,tcag7.1015,tcag7.1017,tcag7.1260,tcag7.23
sample,,,,,,,,,,,,,,,,,,,,,
TCGA.AO.A12F,0.944803,-0.521089,-0.088686,2.161283,-0.504189,-0.393695,0.249436,-0.019315,2.145165,0.221888,...,1.131568,-0.726615,1.868708,0.371048,1.032511,-0.796910,0.430843,2.462129,0.859722,0.573903
TCGA.E2.A14S,-1.435701,0.612919,0.364320,0.235707,-0.444783,0.656335,0.027372,0.188802,0.573184,0.751236,...,0.007255,-0.218478,-0.170075,2.003324,0.367646,-0.335989,-1.058018,0.458436,-0.612609,-0.164528
TCGA.E2.A154,-0.040088,0.560822,-0.318574,-1.008668,1.169428,1.815257,-0.240907,-1.230712,-1.081060,-0.460050,...,1.157267,-1.077419,-0.376896,0.077246,-0.415900,-0.253728,0.041099,1.565449,-0.764582,1.417542
TCGA.E2.A1B1,0.372548,-0.825536,-0.814882,1.245757,1.506113,-1.463759,0.661449,-1.175738,-1.833079,-0.777659,...,-1.130818,1.198008,-1.242092,-1.315932,0.764677,0.979531,-0.901090,-0.643199,0.286347,1.128720
TCGA.AO.A03P,-1.865562,0.609208,1.546514,-0.718382,-0.809749,0.749435,-2.321338,1.213679,0.648119,0.668298,...,0.949241,-0.583338,0.494524,0.965646,0.479029,-0.261562,-0.079536,0.856961,-0.903160,3.797596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA.A8.A08C,0.196853,-0.515379,-0.688128,-1.256729,-1.338724,-0.049577,-0.544825,1.052683,0.626471,0.405981,...,1.998073,-1.146433,0.937307,1.121220,0.490296,1.400627,-0.362048,1.842202,-0.841856,-0.110933
TCGA.AO.A128,-1.308236,0.609922,0.426989,-0.206896,0.986092,-0.885123,0.727637,-0.158714,-1.060078,-0.394042,...,-0.118208,-0.671416,-0.432916,0.575869,0.043905,0.248324,-0.208419,0.794652,0.844782,0.810122
TCGA.BH.A0EB,1.236864,-0.230343,-0.160644,0.842327,0.428551,-0.199245,1.457276,0.477417,0.306746,-0.109652,...,0.558282,-0.190728,0.091351,-1.108172,-0.005134,0.047460,-0.262034,-2.177202,0.013827,-0.292563


In [66]:
cosmic_genes_df = pd.read_csv(f"{data_dir}/cosmic_genes.csv")

In [67]:
cosmic_genes_symbols = set(cosmic_genes_df["Gene Symbol"].to_list())
cosmic_genes_entrez = cosmic_genes_df["Entrez GeneId"].to_list()

In [68]:
tcga_cols = set(X_train_df.columns.to_list())

tcga_cosmic_cols = list(set(tcga_cols & cosmic_genes_symbols))
len(tcga_cosmic_cols)

219

In [69]:
X_train_df, X_test_df = X_train_df[tcga_cosmic_cols], X_test_df[tcga_cosmic_cols]

In [76]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold, KFold
from sklearn.model_selection import cross_val_score

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
from sklearn.linear_model import LogisticRegression

log_param_grid = {"C": np.logspace(-2, 1, 10)}
log_grid_cv = GridSearchCV(estimator=LogisticRegression(max_iter=10000), param_grid=log_param_grid, verbose=3,
                           scoring="roc_auc", cv=cv).fit(X_train_df, y_train_df)
log_grid_cv.best_params_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END ............................C=0.01;, score=0.650 total time=   0.6s
[CV 2/5] END ............................C=0.01;, score=0.586 total time=   0.4s
[CV 3/5] END ............................C=0.01;, score=0.582 total time=   0.7s
[CV 4/5] END ............................C=0.01;, score=0.477 total time=   0.4s
[CV 5/5] END ............................C=0.01;, score=0.549 total time=   0.3s
[CV 1/5] END ............C=0.021544346900318832;, score=0.648 total time=   0.2s
[CV 2/5] END ............C=0.021544346900318832;, score=0.592 total time=   0.6s
[CV 3/5] END ............C=0.021544346900318832;, score=0.582 total time=   0.4s
[CV 4/5] END ............C=0.021544346900318832;, score=0.487 total time=   0.2s
[CV 5/5] END ............C=0.021544346900318832;, score=0.559 total time=   0.6s
[CV 1/5] END ............C=0.046415888336127774;, score=0.653 total time=   0.4s
[CV 2/5] END ............C=0.046415888336127774;

{'C': 0.046415888336127774}

In [77]:
clf = LogisticRegression(max_iter=10000, C=log_grid_cv.best_params_["C"])
cv_score = np.mean(cross_val_score(clf, X_train_df, y_train_df, scoring="roc_auc", cv=cv))
clf.fit(X_train_df, y_train_df)
test_score = roc_auc_score(y_test_df, clf.predict_proba(X_test_df)[:,1])
print(f"cv score: {cv_score}, test_score: {test_score}")

cv score: 0.574810317887241, test_score: 0.5652797704447632


In [52]:
X_train_df["CDH17"]

sample
TCGA.AO.A12F   -1.085519
TCGA.E2.A14S   -1.329566
TCGA.E2.A154   -0.159914
TCGA.E2.A1B1    1.461793
TCGA.AO.A03P   -1.315656
                  ...   
TCGA.A8.A08C   -0.727669
TCGA.AO.A128   -0.142211
TCGA.BH.A0EB    0.153047
TCGA.AO.A03R   -0.778249
TCGA.BH.A0E0   -0.039788
Name: CDH17, Length: 394, dtype: float64